In [ ]:
from mazegen import MazeEnv
import cv2

In [ ]:
env = MazeEnv(size=5,render_mode="human", seed=393939)

state, _ = env.reset()
key_dict={119: 0, 100: 1, 115: 2, 97:3}
done = False
while not done:
    key_press=cv2.waitKey(0)
    state, reward, done, _, info = env.step(key_dict[key_press])
    print("State: \n", state)
    print("Reward: ",reward)
    print("Score: ", env.score)
    print("-----------")
    env.render()

In [1]:
import gymnasium as gym
import random
from dqnAgent import DQN
import numpy as np
from tqdm import tqdm
import tensorflow as tf

model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(50, input_shape=(4,)),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.Dense(2)
        ])

agent=DQN(model)

https://arxiv.org/pdf/1507.04296.pdf

In [6]:
# Exploration
epsilon=1
EPSILON_DECAY=0.99
MIN_EPSILON=0.001

# Visualization
PREVIEW_TRAIN=True
PREVIEW_EVAL=True

# Episodes
N_EPISODES=20000
AGGREGATE_STATS_EVERY=200
LOAD_PATH="best_model_cartpole.keras"

In [7]:
env = gym.make("CartPole-v1", render_mode="human")
test_env=gym.make("CartPole-v1", render_mode="human")
N_ACTION = 2

In [8]:
current_ep_rewards=[]
ep_rewards=[]

if LOAD_PATH != "":
    agent.load(LOAD_PATH)
    
for episode in tqdm(range(N_EPISODES), ascii=True, unit='episodes'):

    # Set reward to initial values
    episode_reward=0

    # Reset Environment
    current_state, _ =env.reset()

    # Iterate until episode end
    done = False
    while not done:

        # We use epsilon greedy policy, we take a random number and compare it to epsilon
        if random.random() > epsilon:
            # Choose action from q table
            action = np.argmax(agent.get_qtable(np.array([current_state])))
        else:
            # Random action
            action = random.choice([i for i in range(N_ACTION)])

        # Perform the step and get data
        new_state, reward, done, truncated, info = env.step(action)
        
        # Sum the reward
        episode_reward+=reward

        # Show preview if conditions are met
        if PREVIEW_TRAIN:
            env.render()

        # Every step we update replay memory
        agent.append_replay_memory((current_state, action, new_state, reward, done))
        agent.train(done)

        current_state = new_state

    current_ep_rewards.append(episode_reward)

    if episode % AGGREGATE_STATS_EVERY == 0:
        print("AGGREGATING EPISODE: ", episode) 
        # Get the average, minimum maximum reward across AGGREGATE_STATS_EVERY episodes
        average_reward = sum(current_ep_rewards) / len(current_ep_rewards)
        min_reward = min(current_ep_rewards)
        max_reward = max(current_ep_rewards)

        ep_rewards.append([average_reward, min_reward, max_reward])
        current_ep_rewards=[]

        # Evaluation
        print("Reward: ", agent.evaluate_policy(test_env, render=PREVIEW_EVAL))
        

    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

  0%|          | 0/20000 [00:00<?, ?episodes/s]

AGGREGATING EPISODE:  0


  0%|          | 0/20000 [00:03<?, ?episodes/s]


KeyboardInterrupt: 

In [5]:
agent.save("best_model.keras")

In [10]:
agent.evaluate_policy(test_env)

206.0

In [ ]:
agent.get_qtable(np.array([env.reset()[0]]))

In [4]:
current_state, _=test_env.reset()
env.render()
total_reward=0

done = False
while not done:
    action=agent.get_qtable(np.array([current_state]))
    action=np.argmax(action)
    current_state, reward, done, _, _ = test_env.step(action)
    total_reward += reward
    env.render()
total_reward

ResetNeeded: Cannot call `env.render()` before calling `env.reset()`, if this is a intended action, set `disable_render_order_enforcing=True` on the OrderEnforcer wrapper.